In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neural_network
import xgboost as xgb


import catboost as cat

import matplotlib.pyplot as plt

In [36]:
df_order_jan = pd.read_csv("orders_before_jan_labeled.csv")
df_order_jan.head()
df_order_jan.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905881 entries, 0 to 905880
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    905881 non-null  object
 1   userID  905881 non-null  int64 
 2   itemID  905881 non-null  int64 
 3   order   905881 non-null  int64 
 4   label   905881 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 34.6+ MB


In [37]:
df_order_jan['label'].value_counts()

0    882569
4      8883
3      6846
2      4522
1      3061
Name: label, dtype: int64

In [38]:
# unbalanced data remove extra 0s 
df_10000_0 = df_order_jan.loc[df_order_jan['label']==0].sample(n=10000, axis=0, random_state=1)
df_balanced_jan = df_10000_0.append(df_order_jan.loc[df_order_jan['label']!=0])
df_balanced_jan['label'].value_counts()

0    10000
4     8883
3     6846
2     4522
1     3061
Name: label, dtype: int64

In [39]:
# load other dataset
df_cat_hierarchy = pd.read_csv('category_hierarchy.csv', delimiter='|',error_bad_lines=False)
df_items = pd.read_csv('items.csv',sep='|',error_bad_lines=False)
# fill NA values in category column -- 4300 is the category with null values
df_items["categories"] = df_items["categories"].fillna('[4300]')
df_items
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32776 entries, 0 to 32775
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   itemID      32776 non-null  int64 
 1   brand       32776 non-null  int64 
 2   feature_1   32776 non-null  int64 
 3   feature_2   32776 non-null  int64 
 4   feature_3   32776 non-null  int64 
 5   feature_4   32776 non-null  int64 
 6   feature_5   32776 non-null  int64 
 7   categories  32776 non-null  object
dtypes: int64(7), object(1)
memory usage: 2.0+ MB


In [40]:
# transform categories from str to int list
df_items['categories'] = df_items['categories'].map(lambda x: list(map(int,x.split("[")[1].split("]")[0].split(","))))

In [41]:
# from categories list generate parent_categories
df_indexed = df_cat_hierarchy.set_index(['category'])
df_indexed

,parent_category
category,
0,75
1,1499
2,1082
3,3498
4,1623
...,...
4295,3898
4296,3898
4297,3898


In [42]:
df_indexed.loc[[2890, 855, 3908, 3909]]['parent_category']

category
2890    2832
855     1178
3908    3898
3909    3898
Name: parent_category, dtype: int64

In [43]:
# set parent of null as null
df_indexed.loc[4300] = [4300]
df_indexed

,parent_category
category,
0,75
1,1499
2,1082
3,3498
4,1623
...,...
4296,3898
4297,3898
4298,3898


In [44]:
# parent_categories generate
df_items['parent_categories'] = df_items['categories'].map(lambda x : df_indexed.loc[x]['parent_category'].tolist())

In [45]:
df_items

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]","[2832, 1178, 3898, 3898]"
1,28640,1366,10,1,537,0,101,[4300],[4300]
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3...","[1420, 3860, 600, 600, 3241, 3241, 3241, 600, ..."
3,21399,1090,10,1,511,0,0,[3270],[1420]
4,8504,768,4,1,484,0,66,[2470],[2566]
...,...,...,...,...,...,...,...,...,...
32771,12036,1383,10,0,503,0,101,"[3502, 1088, 2004, 691, 3422, 2308, 308, 3688,...","[3860, 458, 458, 458, 458, 322, 600, 600, 600,..."
32772,22709,567,4,2,491,3,66,"[3900, 3905, 3903, 3910, 1807, 274, 1467, 914]","[3898, 3898, 3898, 3898, 1072, 2920, 2475, 3565]"
32773,32322,1385,10,1,500,0,37,"[2686, 2609, 725]","[2917, 1175, 725, 1735]"
32774,19118,1113,6,2,491,0,117,"[3900, 3910, 3903, 2473, 914, 274]","[3898, 3898, 3898, 2566, 3565, 2920]"


In [46]:
# Joining df_orders_jan dataframe and df_items dataframe
merge_jan = pd.merge(df_items, df_balanced_jan, how='inner', on='itemID')
#change the date coulumn's datayte to datetime datatype
merge_jan['date'] = pd.to_datetime(merge_jan['date'])
merge_jan = merge_jan.set_index('date')

In [47]:
merge_jan_no_cat = merge_jan.drop(columns=['categories', 'parent_categories'])

In [48]:
y = merge_jan_no_cat['label']
X = merge_jan_no_cat.drop(columns='label')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
X_train
X_test

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,userID,order
date,,,,,,,,,
2020-08-11,8475,369,10,3,-1,-1,-1,16451,1
2020-08-11,31800,1040,4,3,452,0,12,25454,2
2020-11-29,32528,1260,4,0,162,3,53,12551,1
2020-08-01,21163,539,10,1,345,0,95,9884,5
2020-12-14,12101,453,4,0,458,0,-1,36035,6
...,...,...,...,...,...,...,...,...,...
2020-07-03,26503,618,4,0,71,0,53,42623,1
2020-08-23,18154,400,10,0,261,3,84,38564,1
2020-10-25,31615,504,10,0,441,3,84,45428,1


In [49]:
X_test

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,userID,order
date,,,,,,,,,
2020-08-11,8475,369,10,3,-1,-1,-1,16451,1
2020-08-11,31800,1040,4,3,452,0,12,25454,2
2020-11-29,32528,1260,4,0,162,3,53,12551,1
2020-08-01,21163,539,10,1,345,0,95,9884,5
2020-12-14,12101,453,4,0,458,0,-1,36035,6
...,...,...,...,...,...,...,...,...,...
2020-07-03,26503,618,4,0,71,0,53,42623,1
2020-08-23,18154,400,10,0,261,3,84,38564,1
2020-10-25,31615,504,10,0,441,3,84,45428,1


In [50]:
y_train

date
2020-11-23    4
2020-07-11    3
2020-10-03    0
2020-12-14    0
2020-11-23    0
             ..
2020-07-16    3
2020-08-10    4
2020-12-28    4
2020-11-11    3
2020-11-02    2
Name: label, Length: 26649, dtype: int64

In [51]:
y_test

date
2020-08-11    0
2020-08-11    0
2020-11-29    4
2020-08-01    3
2020-12-14    4
             ..
2020-07-03    4
2020-08-23    3
2020-10-25    4
2020-12-18    4
2020-10-13    0
Name: label, Length: 6663, dtype: int64

In [52]:
# upload submission dataset
sub_jan = pd.read_csv("submission_jan.csv", sep="|")
sub_jan

,userID,itemID,prediction
0,0,20664,NaN
1,0,28231,NaN
2,13,2690,NaN
3,15,1299,NaN
4,15,20968,NaN
...,...,...,...
9935,46118,20106,NaN
9936,46124,19677,NaN
9937,46125,12878,NaN
9938,46127,7963,NaN


In [53]:
# baseline(didn't consider time series)--xgboost
def xgb_train(X_train, y_train, X_test, y_test, verbose=True):
    model_xgb = xgb.XGBClassifier(
        max_depth=10, # raw8
        n_estimators=1000,
        min_child_weight=300, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3,    
        seed=42        
    )

    model_xgb.fit(
        X_train, 
        y_train,
        eval_metric='auc',
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=verbose,
        early_stopping_rounds=10 
    )
    print(model_xgb.best_score)
    return model_xgb

In [54]:
model_xgb = xgb_train(X_train, y_train, X_test, y_test, verbose=False)

0.621942


In [55]:
pre_merge = pd.merge(sub_jan.drop(columns='prediction'), X, on=['userID','itemID'], how="inner")
prediction = model_xgb.predict(pre_merge)
sub_xgb = sub_jan.copy()
sub_xgb['prediction'] = pd.Series(prediction)
# user and items haven't shown up predict 0
sub_xgb['prediction'] = sub_xgb['prediction'].fillna(0)
# submission.to_csv('submission_xgb.csv', index=False)
sub_xgb['prediction'].value_counts()

0.0    8406
4.0    1265
3.0     269
Name: prediction, dtype: int64

In [25]:
sub_xgb.to_csv('submission_xgb.csv')

In [56]:
# baseline1: fill 1
sub_jan_all1 = sub_jan.copy()
sub_jan_all1['prediction'] = sub_jan['prediction'].fillna(1)
sub_jan_all1.to_csv('submission_all1.csv')

Baseline model of Naive,decisiontree,KNN with Default  and corresponding hyperpparamters

In [57]:
def baseline(classifier,x_train_dt,x_test_dt,y_train_dt,y_test_dt):
    baseline=classifier
    baseline.fit(x_train_dt,y_train_dt)
    stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    pred=baseline.predict(x_test_dt)
    print('CV score with default parameters:{}'.format(cross_val_score(classifier,x_train_dt, y_train_dt, cv=stratified_10_fold_cv, scoring='accuracy').mean()))
    print('classification_report on test set with default parameters:\n')
    print(classification_report(y_test_dt,pred))
    # cnf_matrix = confusion_matrix(y_test_dt,pred)    
    #np.set_printoptions(precision=2)
   # plot_confusion_matrix(cnf_matrix,classes=labels,title='confusion matrix:default')

In [58]:
def naivebayes (x_train,x_test,y_train,y_test):
    gnb=GaussianNB()
    gnb.fit(x_train, y_train)
    stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracy=cross_val_score(gnb,x_train,y_train,cv=cv)
    print('Mean CV score is: {}'.format(accuracy.mean()))
    params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
    grid_search = GridSearchCV( estimator=gnb,param_grid=params_NB, cv=cv,verbose=1, scoring='accuracy') 
    grid_search.fit(x_train, y_train)
    print("The best parameter is:{}".format(grid_search.best_params_))
    print("The best validation score is:{}".format(grid_search.best_score_))
    gnb.set_params(var_smoothing=grid_search.best_params_)
    prediction = gnb.predict(x_test)
    print(classification_report(y_test, prediction))
    cm=confusion_matrix(y_test,prediction)         

In [59]:
baseline(GaussianNB(),X_train,X_test,y_train,y_test)
print('\n----------after hyperparameter tuning----------\n')
naivebayes(X_train,X_test,y_train, y_test)

CV score with default parameters:0.3137075959636936
classification_report on test set with default parameters:

              precision    recall  f1-score   support

           0       0.31      0.96      0.47      1995
           1       0.00      0.00      0.00       631
           2       0.00      0.00      0.00       916
           3       0.14      0.00      0.00      1366
           4       0.28      0.09      0.14      1755

    accuracy                           0.31      6663
   macro avg       0.15      0.21      0.12      6663
weighted avg       0.20      0.31      0.18      6663


----------after hyperparameter tuning----------

Mean CV score is: 0.3137075959636936
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
The best parameter is:{'var_smoothing': 1e-09}
The best validation score is:0.3137075959636936
              precision    recall  f1-score   support

           0       0.31      0.96      0.47      1995
           1       0.00      0.00      0.00

In [60]:
def decisionTree(x_train,x_test,y_train,y_test):
    
    parameters={
            'criterion':['gini','entropy'],
            'max_depth':[2,3,4,5,6,7,8]
            }
    dtree=DecisionTreeClassifier()
    stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    grid_search=GridSearchCV(dtree,parameters,scoring='accuracy',cv=stratified_10_fold_cv)
    grid_search.fit(x_train,y_train) 
    print('CV score with best parameters:{}'.format(cross_val_score(grid_search.best_estimator_,x_train, y_train, cv=stratified_10_fold_cv, scoring='accuracy').mean()))
    print('best parameters:{}'.format(grid_search.best_params_))
    pred=grid_search.predict(x_test)
    print('classification_report with best parameters:\n')
    print(classification_report(y_test,pred))
    #confusion matrix
    dt_cnf_matrix = confusion_matrix(y_test,pred)    
    np.set_printoptions(precision=2)    
    
    return grid_search.best_estimator_

In [61]:
baseline(DecisionTreeClassifier(),X_train,X_test,y_train,y_test)
print('\n----------after hyperparameter tuning----------\n')
dtree=decisionTree(X_train,X_test,y_train, y_test)

CV score with default parameters:0.30620315906901274
classification_report on test set with default parameters:

              precision    recall  f1-score   support

           0       0.44      0.40      0.42      1995
           1       0.17      0.18      0.18       631
           2       0.21      0.23      0.22       916
           3       0.25      0.27      0.26      1366
           4       0.34      0.33      0.33      1755

    accuracy                           0.31      6663
   macro avg       0.28      0.28      0.28      6663
weighted avg       0.32      0.31      0.31      6663


----------after hyperparameter tuning----------

CV score with best parameters:0.35273350742862936
best parameters:{'criterion': 'gini', 'max_depth': 8}
classification_report with best parameters:

              precision    recall  f1-score   support

           0       0.40      0.67      0.50      1995
           1       0.10      0.01      0.01       631
           2       0.34      0.05   

In [76]:
def KNN(x_train,x_test,y_train,y_test):
    
    parameters={
           'n_neighbors' : list(range(1,30)),
        'p': [1,2],
        'weights' : ["uniform", "distance"]
            }
    knn_2 = KNeighborsClassifier()
    stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    Grid_search_knn = GridSearchCV(knn_2, parameters, cv=stratified_10_fold_cv)
    best_model = Grid_search_knn.fit(x_test,y_test)
    #Print The value of best Hyperparameters
    cv_score=cross_val_score(Grid_search_knn.best_estimator_,x_train, y_train, cv=stratified_10_fold_cv, scoring='accuracy').mean()
    print("Mean CV score is {} with params {}".format(cv_score, Grid_search_knn.best_params_))
    pred=Grid_search_knn.predict(x_test)
    #print('classification_report on test set with best parameters:\n')
   # print(classification_report(y_test,pred))
  
    np.set_printoptions(precision=2)
   
    return Grid_search_knn.best_estimator_

In [77]:
baseline(KNeighborsClassifier(),X_train,X_test,y_train,y_test)
print('\n----------after hyperparameter tuning----------\n')
knn=KNN(X_train,X_test,y_train, y_test)

CV score with default parameters:0.29502071959389037
classification_report on test set with default parameters:

              precision    recall  f1-score   support

           0       0.36      0.51      0.43      1995
           1       0.18      0.11      0.14       631
           2       0.23      0.20      0.21       916
           3       0.25      0.23      0.24      1366
           4       0.31      0.25      0.28      1755

    accuracy                           0.30      6663
   macro avg       0.27      0.26      0.26      6663
weighted avg       0.29      0.30      0.29      6663


----------after hyperparameter tuning----------

Mean CV score is 0.32995598318769054 with params {'n_neighbors': 26, 'p': 1, 'weights': 'distance'}
classification_report on test set with best parameters:

